<a href="https://colab.research.google.com/github/nMishelRamirez/Project_IR/blob/main/ProyectoIR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto de Recuperación de la información

Integrantes:
- Ramirez Mishel
- Zaldumbide Danna

In [ ]:
# Conexión con el Script
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install contractions
!pip install ir_datasets
!pip install rank_bm25

In [ ]:
# Importar la librería
import sys
sys.path.append('/content/drive/MyDrive')
import json
import gradio as gr
import numpy as np
import pandas as pd
import nltk
import pickle
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score, f1_score
from rank_bm25 import BM25Okapi

# Construcción del índice

## Cargar corpus.jsonl

In [ ]:
# Cargar corpus.jsonl
corpus_path = '/content/drive/MyDrive/ir_datasets/gaming/corpus.jsonl'
docs = []

with open(corpus_path, 'r') as f:
    for line in f:
        data = json.loads(line)
        docs.append({
            'doc_id': data['_id'],
            'text': data['text']
        })

# Crear DataFrame
corpus_df = pd.DataFrame(docs)

# Extraer textos
corpus = [doc['text'] for doc in docs]


## Preprocesamiento

In [ ]:
# import preprocessing as pp
from preprocessing import preprocess_dataframe

try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    nltk.download('punkt_tab')

# Crear un DataFrame
df = pd.DataFrame({'raw': corpus})

# Aplicar preprocesamiento
processed_df = preprocess_dataframe(df, lang='english', return_all=True)

# Mostrar el resultado
print(processed_df[['raw', 'tokenized', 'normalized', 'filtered', 'stemmed', 'lemmatized']].head())

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


                                                 raw  \
0  What's your Supreme Commander 2 build order. I...   
1  You can view how many bone shards, paintings a...   
2  Does that mean that if no one screws up and ev...   
3  I have some clean rags that I found in a first...   
4  I know of the extra class slots and the variou...   

                                           tokenized  \
0  [What, is, your, Supreme, Commander, 2, build,...   
1  [You, can, view, how, many, bone, shards, ,, p...   
2  [Does, that, mean, that, if, no, one, screws, ...   
3  [I, have, some, clean, rags, that, I, found, i...   
4  [I, know, of, the, extra, class, slots, and, t...   

                                          normalized  \
0  [what, is, your, supreme, commander, build, or...   
1  [you, can, view, how, many, bone, shards, pain...   
2  [does, that, mean, that, if, no, one, screws, ...   
3  [i, have, some, clean, rags, that, i, found, i...   
4  [i, know, of, the, extra, class, slots, and

### Resultados del preprocesamiento

In [ ]:
processed_df

,raw,tokenized,normalized,filtered,stemmed,lemmatized
0,What's your Supreme Commander 2 build order. I...,"[What, is, your, Supreme, Commander, 2, build,...","[what, is, your, supreme, commander, build, or...","[supreme, commander, build, order, want, mass,...","[suprem, command, build, order, want, mass, ex...","[supreme, commander, build, order, want, mass,..."
1,"You can view how many bone shards, paintings a...","[You, can, view, how, many, bone, shards, ,, p...","[you, can, view, how, many, bone, shards, pain...","[view, many, bone, shards, paintings, runes, f...","[view, mani, bone, shard, paint, rune, found, ...","[view, many, bone, shard, painting, rune, foun..."
2,Does that mean that if no one screws up and ev...,"[Does, that, mean, that, if, no, one, screws, ...","[does, that, mean, that, if, no, one, screws, ...","[mean, one, screws, everybody, gets, daily, ra...","[mean, one, screw, everybodi, get, daili, rati...","[mean, one, screw, everybody, get, daily, rati..."
3,I have some clean rags that I found in a first...,"[I, have, some, clean, rags, that, I, found, i...","[i, have, some, clean, rags, that, i, found, i...","[clean, rags, found, first, aid, kit, would, l...","[clean, rag, found, first, aid, kit, would, li...","[clean, rag, found, first, aid, kit, would, li..."
4,I know of the extra class slots and the variou...,"[I, know, of, the, extra, class, slots, and, t...","[i, know, of, the, extra, class, slots, and, t...","[know, extra, class, slots, various, cosmetic,...","[know, extra, class, slot, variou, cosmet, stu...","[know, extra, class, slot, various, cosmetic, ..."
...,...,...,...,...,...,...
45296,I know you can't go to an 85 or 90 dungeon and...,"[I, know, you, can, not, go, to, an, 85, or, 9...","[i, know, you, can, not, go, to, an, or, dunge...","[know, go, dungeon, expect, get, rep, even, ta...","[know, go, dungeon, expect, get, rep, even, ta...","[know, go, dungeon, expect, get, rep, even, ta..."
45297,Interrogating a sectoid resulted in my earning...,"[Interrogating, a, sectoid, resulted, in, my, ...","[interrogating, a, sectoid, resulted, in, my, ...","[interrogating, sectoid, resulted, earning, re...","[interrog, sectoid, result, earn, research, cr...","[interrogating, sectoid, resulted, earning, re..."
45298,"In Assassin's Creed: Revelations, what is the ...","[In, Assassin, 's, Creed, :, Revelations, ,, w...","[in, assassin, creed, revelations, what, is, t...","[assassin, creed, revelations, highest, level,...","[assassin, creed, revel, highest, level, assas...","[assassin, creed, revelation, highest, level, ..."
45299,A friend of mine didn't realise you had to do ...,"[A, friend, of, mine, did, not, realise, you, ...","[a, friend, of, mine, did, not, realise, you, ...","[friend, mine, realise, something, order, pres...","[friend, mine, realis, someth, order, prestig,...","[friend, mine, realise, something, order, pres..."


## Construcción del índice invertido

In [ ]:
# Crear índice invertido
inverted_index = defaultdict(dict)

# Recorrer documentos lematizados
for doc_id, tokens in enumerate(processed_df['lemmatized']):

    # Contar la frecuencia de cada término en el documento actual
    term_freq = defaultdict(int)
    for token in tokens:
        term_freq[token] += 1

    # Agregar las frecuencias de los términos al índice invertido
    for term, freq in term_freq.items():
        inverted_index[term][doc_id] = freq


### Resultados

In [ ]:
# Imprimir las primeras 20 entradas del índice invertido
for i, term in enumerate(sorted(inverted_index.keys())):
    if i >= 20:  # Imprimir solo los 20 primeros
        break
    print(f"Término: {term}")
    print(f"Posting list: {inverted_index[term]}")
    print("---")


Término: aa
Posting list: {825: 3, 1646: 3, 3152: 1, 3616: 1, 4145: 1, 5668: 1, 8197: 1, 9613: 1, 10154: 1, 10645: 1, 11107: 1, 11639: 1, 12078: 3, 12769: 2, 13973: 1, 14519: 1, 16239: 1, 18528: 3, 18542: 2, 18944: 1, 19122: 1, 19709: 2, 22045: 1, 22084: 2, 22839: 1, 23382: 1, 24237: 1, 24882: 2, 27000: 1, 27672: 1, 29446: 1, 30103: 1, 30366: 1, 30712: 1, 31523: 1, 31649: 2, 36153: 1, 38216: 1, 39672: 2, 40418: 1, 41075: 1, 42391: 1, 43946: 1, 44879: 1}
---
Término: aaa
Posting list: {20010: 1, 26700: 3, 34793: 2, 38379: 1}
---
Término: aaaaaa
Posting list: {6384: 1}
---
Término: aaaaaand
Posting list: {34959: 1}
---
Término: aaaaages
Posting list: {41538: 1}
---
Término: aaaae
Posting list: {32527: 1}
---
Término: aaaand
Posting list: {14676: 1, 35210: 1}
---
Término: aaages
Posting list: {20446: 1}
---
Término: aaagggghhh
Posting list: {30741: 2}
---
Término: aaah
Posting list: {39398: 1}
---
Término: aaand
Posting list: {24826: 1}
---
Término: aabb
Posting list: {236: 1, 294: 1, 125

In [ ]:
# Construir lista de filas
rows = []
for term, postings in inverted_index.items():
    for doc_id, freq in postings.items():
        rows.append({'term': term, 'doc_id': doc_id, 'frequency': freq})

# Crear DataFrame
inverted_index_df = pd.DataFrame(rows)

# Ver resultados
inverted_index_df


,term,doc_id,frequency
0,supreme,0,1
1,supreme,1285,1
2,supreme,4858,1
3,supreme,4920,2
4,supreme,5615,1
...,...,...,...
1289737,loitering,45290,1
1289738,easydrive,45290,1
1289739,borealis,45292,1
1289740,komodo,45293,2


# Modelo de recuperación

## TF-IDF

In [ ]:
# Unir tokens lematizados en strings
processed_df['lemmatized_text'] = processed_df['lemmatized'].apply(lambda x: " ".join(x))

# Crear matriz TF-IDF
vectorizer = TfidfVectorizer()
corpus_tfidf = vectorizer.fit_transform(processed_df['lemmatized_text'])

# Lista de términos del vocabulario
tfidf_terms = vectorizer.get_feature_names_out()


In [ ]:
def consultar_tfidf(query, top_k=5):
    # Preprocesar la query
    query_tokens = preprocess_dataframe(pd.DataFrame({'raw': [query]}), lang='english', return_all=True)['lemmatized'][0]
    query_text = " ".join(query_tokens)

    # Vectorizar la query
    query_vec = vectorizer.transform([query_text])

    # Calcular similitud coseno
    scores = cosine_similarity(query_vec, corpus_tfidf)[0]

    # Obtener top-k documentos
    top_indices = scores.argsort()[::-1][:top_k]
    resultados = processed_df.iloc[top_indices][['raw']].copy()
    resultados['score'] = scores[top_indices]

    return resultados


## BM25

In [ ]:
# Corpus tokenizado para BM25
tokenized_corpus = processed_df['lemmatized'].tolist()
bm25 = BM25Okapi(tokenized_corpus)

def consultar_bm25(query, top_k=5):
    # Preprocesar query
    query_tokens = preprocess_dataframe(pd.DataFrame({'raw': [query]}), lang='english', return_all=True)['lemmatized'][0]

    # Obtener scores BM25
    scores = bm25.get_scores(query_tokens)

    top_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_k]
    resultados = processed_df.iloc[top_indices][['raw']].copy()
    resultados['score'] = [scores[i] for i in top_indices]

    return resultados

In [ ]:
query = "Is it safe to use the Abandoned Shack for storage?"
print("Top resultados con TF-IDF:")
print(consultar_tfidf(query))

print("\nTop resultados con BM25:")
print(consultar_bm25(query))


Top resultados con TF-IDF:
                                                     raw     score
17939  There is a shack you can buy next to Onett whi...  0.313622
3042   * dota_disable_range_finder    * dota_range_di...  0.273985
43286  Has anyone found any abandoned ships in Albion...  0.273874
19881  It's been fairly common knowledge that the onl...  0.264949
23275  I haven't tried it because I'm afraid this kin...  0.263520

Top resultados con BM25:
                                                     raw      score
4713   In Skyrim, what house or safe storage chest is...  17.794805
19881  It's been fairly common knowledge that the onl...  17.781195
39977  It says in the Wiki that both the Piggy Bank a...  14.552183
19781  I read, once upon a time, that plastic pages c...  13.685107
9476   My storage is full and there's so many nice th...  13.489387


In [ ]:
# ============================ #
#   VECTORIZACIÓN             #
# ============================ #

# TF-IDF Vectorizer
def tfidf_vectorizer(corpus):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(corpus)
    return X, vectorizer

# BM25 Vectorizer
def bm25_vectorizer(corpus):
    tokenized_docs = [doc.split() for doc in corpus]
    bm25 = BM25Okapi(tokenized_docs)
    return bm25

# ============================ #
#   FUNCIÓN DE BÚSQUEDA        #
# ============================ #
def search(query, method='TF-IDF', top_n=5, return_df=False):
    corpus_text = [' '.join(doc) for doc in processed_df['lemmatized']]

    # Preprocesar consulta
    query_tokens = preprocess_dataframe(pd.DataFrame({'raw': [query]}), lang='english', return_all=True)['lemmatized'][0]

    if method == 'BM25':
        bm25 = bm25_vectorizer(corpus_text)
        scores = bm25.get_scores(query_tokens)
    else:
        X_tfidf, vectorizer = tfidf_vectorizer(corpus_text)
        query_tfidf = vectorizer.transform([' '.join(query_tokens)])
        scores = cosine_similarity(query_tfidf, X_tfidf).flatten()

    ranked_indices = np.argsort(scores)[::-1][:top_n]

    results = []
    for idx in ranked_indices:
        doc = processed_df.iloc[idx]
        results.append({
            'Doc_ID': int(processed_df.index[idx]),
            'Text': ' '.join(doc['lemmatized']),
            'Score': scores[idx]
        })

    if return_df:
        return pd.DataFrame(results)
    else:
        return "\n".join(
            [f"Rank {i+1}: {row['Doc_ID']} - Score: {row['Score']:.4f}\nText: {row['Text']}\n" for i, row in enumerate(results)]
        )

# Interfaz básica

# Evaluación de resultados